# Importações

In [11]:
import pandas as pd
import numpy as np

# Exploratória

In [12]:
df_raw = pd.read_csv('./data/raw/raw_data.csv')
df_raw.head(5)

,id,created_at,updated_at,deleted_at,name_hash,email_hash,address_hash,birth_date,status,version,city,state,neighborhood,last_date_purchase,average_ticket,items_quantity,all_revenue,all_orders,recency,marketing_source
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,08/15/17 07:05 AM,01/14/21 11:23 AM,NaN,312d206168a318614897e8ccac43bff9,83eb3aed9a44377df80ce876dce92c9a,8b4bfaa0cbc41a16f46da15ddcd6a907,07/10/74 12:00 AM,active,2.31.7,Peixoto da Praia,AM,Aparecida 7ª Seção,01/14/21 11:23 AM,151.142942,10,906.857651,6,35,crm
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,12/31/19 09:53 PM,01/08/21 11:23 AM,NaN,de448fcb47d0d6a873b2eef52b5ee595,72678bb35e2ac84ed373e81dd9dca28c,22f1cfa1847f38da3f3cb114dd2b9247,07/06/40 12:00 AM,paused,3.30.12,Fernandes,RR,Santa Isabel,01/08/21 11:23 AM,236.991790,4,236.991790,1,41,organic_search
2,dc067cd2-c021-42bd-8c0e-beb267280e66,03/07/19 11:46 PM,01/07/21 11:23 AM,NaN,cb09e447ddc38283373d56bb46498e6a,668f4ee9add29c7bd02c485f1b7509e3,6cb47446a086ee6483b3eb954f11467a,03/18/63 12:00 AM,active,3.28.9,Lopes,RR,Estrela,01/07/21 11:23 AM,211.955597,13,2331.511572,11,42,organic_search
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,07/21/18 10:17 AM,01/10/21 11:23 AM,NaN,52593437a405b11b3557170680ef80c8,d3fb45188d95c8d7cc49da5b4f727c86,0a6f0c54db1e6f19347f96b50f8092a4,11/21/80 12:00 AM,active,3.34.3,Campos do Campo,PE,Confisco,01/10/21 11:23 AM,204.113227,8,1224.679359,6,39,organic_search
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,06/08/18 12:09 PM,01/18/21 11:23 AM,NaN,dbda4b778a966c21904238ed2d2005db,a0f76bc49b4c43327b536da6e1a1465e,143b9f169b4fa1692f6d79b5682169b5,07/07/59 12:00 AM,active,3.19.8,das Neves,RJ,Vila Suzana Segunda Seção,01/18/21 11:23 AM,252.940997,9,2023.527980,8,31,crm


In [13]:
df_raw.isna().sum()

id                       0
created_at               0
updated_at               0
deleted_at            9495
name_hash                0
email_hash               0
address_hash             0
birth_date               0
status                   0
version                  0
city                     0
state                    0
neighborhood             0
last_date_purchase       0
average_ticket           0
items_quantity           0
all_revenue              0
all_orders               0
recency                  0
marketing_source         0
dtype: int64

In [14]:
df_raw.dtypes

id                     object
created_at             object
updated_at             object
deleted_at             object
name_hash              object
email_hash             object
address_hash           object
birth_date             object
status                 object
version                object
city                   object
state                  object
neighborhood           object
last_date_purchase     object
average_ticket        float64
items_quantity          int64
all_revenue           float64
all_orders              int64
recency                 int64
marketing_source       object
dtype: object

# Transformação

## Funções

In [15]:
def obj_to_datetime(col):
     return pd.to_datetime(col, format='%m/%d/%y %H:%M %p')

def diferenca_datas(series_final, series_inicial, periodo):
     return ((series_final - series_inicial) / np.timedelta64(1, periodo))


Uma vez que, de acordo com os padrões POSIX e ISO C, inteiros menores do que 69 são mapeados como sendo anos do século XXI, necessário adequar a idade de nascidos no século XX.

*Fonte*: https://docs.python.org/3/library/time.html

In [16]:
def corrige_idade(array1):
    return array1.where(array1 >= 0, 100 + array1)

## Tabela por usuário

In [17]:
df_usuario = pd.DataFrame()

In [18]:
df_usuario['id'] = df_raw['id']

### Cálculo de tempo de clientela - Meses

In [19]:
aux = pd.DataFrame()
aux['criacao'] = obj_to_datetime(df_raw['created_at'])
aux['exclusao'] = obj_to_datetime(df_raw['deleted_at'])
aux['duracao'] = diferenca_datas(aux.exclusao, aux.criacao, 'M')

In [20]:
aux['duracao'] = aux['duracao'].fillna(-1)
aux['duracao'] = aux['duracao'].astype(int)

In [21]:
# Conforme colunas 'recency' e 'last_update', o arquivo data de 17 de fevereiro de 2021

aux['ultima_aquisicao'] = obj_to_datetime(df_raw['last_date_purchase'])
aux['ultima_aquisicao'] = obj_to_datetime(df_raw['last_date_purchase'])
aux['intervalo_de_aquisicao'] = ((np.where(aux['exclusao'].notna(), aux['exclusao'], np.datetime64("2021-02-17")) - aux.ultima_aquisicao) / np.timedelta64(1, 'M'))
aux['intervalo_de_aquisicao'] = aux['intervalo_de_aquisicao'].astype(int)

In [22]:
df_usuario[['usuario_criado', 
            'usuario_excluido', 
            'meses_assinatura', 
            'intervalo_de_inatividade']] = aux[['criacao', 
                                                'exclusao', 
                                                'duracao', 
                                                'intervalo_de_aquisicao']]
df_usuario

,id,usuario_criado,usuario_excluido,meses_assinatura,intervalo_de_inatividade
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,2017-08-15 07:05:00,NaT,-1,1
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,2019-12-31 09:53:00,NaT,-1,1
2,dc067cd2-c021-42bd-8c0e-beb267280e66,2019-03-07 11:46:00,NaT,-1,1
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,2018-07-21 10:17:00,NaT,-1,1
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,2018-06-08 12:09:00,NaT,-1,0
...,...,...,...,...,...
9995,90c9fd5f-80d2-4f59-9970-6b3e6dac5f57,2017-09-12 12:35:00,NaT,-1,0
9996,0edb7aec-8816-4904-bcd7-cbe141391441,2018-04-20 07:43:00,NaT,-1,0
9997,30130652-d616-4b40-adbb-189e3351016c,2017-08-14 12:28:00,NaT,-1,1
9998,37b52952-afdd-425a-9149-4012ea48ac87,2017-01-19 07:19:00,NaT,-1,1


In [23]:
del aux

### Cálculo de Idade

In [24]:
aux = pd.DataFrame()

aux['nascimento'] = obj_to_datetime(df_raw['birth_date'])
aux['idade'] = diferenca_datas(np.datetime64("2021-02-17"), aux.nascimento, 'Y')
aux['idade'] = corrige_idade(aux['idade']).astype(int)

In [25]:
bins = [0, 30, 40, 50, 60, 70, 120]
labels = [f'{i+1}-{i+10}' for i in bins[:7]]
labels[0], labels[6] = ('< 30', '70 +')
aux['faixa_etaria_10'] = pd.cut(aux['idade'], bins=bins, labels=['< 30', '31-40', '41-50', '51-60', '61-70', '70 +'])

In [26]:
bins = [0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 120]
labels = [f'{i+1}-{i+5}' for i in bins[:12]]
labels[0], labels[11] = ('< 25', '75 +')
aux['faixa_etaria_5'] = pd.cut(aux['idade'], bins=bins, labels=labels)

In [27]:
df_usuario[['idade', 'faixa_etaria_05', 'faixa_etaria_10']] = aux[['idade', 'faixa_etaria_5', 'faixa_etaria_10']]

In [28]:
del aux

### Binarização de status

In [29]:
df_usuario['ativo'] = np.where(df_raw.status == 'canceled', 0, 1)

### Demais

In [30]:
df_usuario[['versao', 'cidade', 'estado', 'gasto_medio', 'media_de_itens', 'gasto_total', 'total_de_pedidos', 'canal_marketing']] = df_raw[['version', 'city', 'state', 'average_ticket', 'items_quantity', 'all_revenue', 'all_orders', 'marketing_source']]
df_usuario

,id,usuario_criado,usuario_excluido,meses_assinatura,intervalo_de_inatividade,idade,faixa_etaria_05,faixa_etaria_10,ativo,versao,cidade,estado,gasto_medio,media_de_itens,gasto_total,total_de_pedidos,canal_marketing
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,2017-08-15 07:05:00,NaT,-1,1,46,46-50,41-50,1,2.31.7,Peixoto da Praia,AM,151.142942,10,906.857651,6,crm
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,2019-12-31 09:53:00,NaT,-1,1,80,75 +,70 +,1,3.30.12,Fernandes,RR,236.991790,4,236.991790,1,organic_search
2,dc067cd2-c021-42bd-8c0e-beb267280e66,2019-03-07 11:46:00,NaT,-1,1,57,56-60,51-60,1,3.28.9,Lopes,RR,211.955597,13,2331.511572,11,organic_search
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,2018-07-21 10:17:00,NaT,-1,1,40,36-40,31-40,1,3.34.3,Campos do Campo,PE,204.113227,8,1224.679359,6,organic_search
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,2018-06-08 12:09:00,NaT,-1,0,61,61-65,61-70,1,3.19.8,das Neves,RJ,252.940997,9,2023.527980,8,crm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,90c9fd5f-80d2-4f59-9970-6b3e6dac5f57,2017-09-12 12:35:00,NaT,-1,0,59,56-60,51-60,1,4.30.12,Souza,RN,215.079785,5,1075.398927,5,paid_search
9996,0edb7aec-8816-4904-bcd7-cbe141391441,2018-04-20 07:43:00,NaT,-1,0,51,51-55,51-60,1,4.28.13,Silveira,AL,254.525826,10,2290.732434,9,organic_search
9997,30130652-d616-4b40-adbb-189e3351016c,2017-08-14 12:28:00,NaT,-1,1,61,61-65,61-70,1,3.18.11,Martins,SC,208.741902,8,1252.451411,6,paid_search
9998,37b52952-afdd-425a-9149-4012ea48ac87,2017-01-19 07:19:00,NaT,-1,1,63,61-65,61-70,1,5.51.6,Gomes das Flores,PA,211.174796,9,0.000000,0,direct


### Salvar arquivo

In [31]:
df_usuario.to_csv('./data/transformed/user_data.csv', sep=',')

## Tabela por datas

### Base

In [32]:
df_data = pd.DataFrame()
df_data[['data_inicio', 'data_fim', 'estado']] = df_raw[['created_at', 'deleted_at', 'state']]
df_data['data_inicio'] = obj_to_datetime(df_data['data_inicio']).dt.strftime('%Y%m')
df_data['data_fim'] = obj_to_datetime(df_data['data_fim'])
df_data['data_fim'] = df_data['data_fim'].fillna(pd.to_datetime('now')).dt.strftime('%Y%m')
df_data['ano_mes'] = df_data['data_inicio']
df_data

,data_inicio,data_fim,estado,ano_mes
0,201708,202305,AM,201708
1,201912,202305,RR,201912
2,201903,202305,RR,201903
3,201807,202305,PE,201807
4,201806,202305,RJ,201806
...,...,...,...,...
9995,201709,202305,RN,201709
9996,201804,202305,AL,201804
9997,201708,202305,SC,201708
9998,201701,202305,PA,201701


### Contas criadas no mês

In [33]:
df_data_aux = df_data[['estado', 'ano_mes']]

In [34]:
df_data_aux['contas_criadas'] = df_data_aux.groupby(['estado', 'ano_mes'])['ano_mes'].transform('count')
df_data_aux = df_data_aux.drop_duplicates()
df_data_aux

C:\Users\adm\AppData\Local\Temp\ipykernel_1220\1402236487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_aux['contas_criadas'] = df_data_aux.groupby(['estado', 'ano_mes'])['ano_mes'].transform('count')


,estado,ano_mes,contas_criadas
0,AM,201708,6
1,RR,201912,8
2,RR,201903,3
3,PE,201807,8
4,RJ,201806,9
...,...,...,...
8696,GO,201707,2
8807,DF,201612,2
8854,MA,202002,2
8945,AC,201805,2


### Contas excluídas no mês

In [35]:
df_data_aux2 = df_data[['estado', 'data_fim']]
df_data_aux2

,estado,data_fim
0,AM,202305
1,RR,202305
2,RR,202305
3,PE,202305
4,RJ,202305
...,...,...
9995,RN,202305
9996,AL,202305
9997,SC,202305
9998,PA,202305


In [36]:
df_data_aux2['contas_excluidas'] = df_data_aux2.groupby(['estado', 'data_fim'])['data_fim'].transform('count')
df_data_aux2 = df_data_aux2[df_data_aux2['data_fim'] != '202304'].drop_duplicates()
df_data_aux2.rename(columns={'data_fim': 'ano_mes'}, inplace=True)

C:\Users\adm\AppData\Local\Temp\ipykernel_1220\3085568216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_aux2['contas_excluidas'] = df_data_aux2.groupby(['estado', 'data_fim'])['data_fim'].transform('count')


### União das colunas para cálculo do total

In [37]:
df_merged = pd.merge(df_data_aux, df_data_aux2, on=['ano_mes', 'estado'], how='outer')
df_merged.fillna(0, inplace=True)
df_merged.sort_values(['estado', 'ano_mes'], inplace=True)

### Cálculos

In [38]:
df_merged['contas_criadas_acumuladas'] = df_merged.groupby(['estado'])['contas_criadas'].cumsum()
df_merged['contas_excluidas_acumuladas'] = df_merged.groupby(['estado'])['contas_excluidas'].cumsum()
df_merged['contas_ativas_fim_mes'] = df_merged['contas_criadas_acumuladas'] - df_merged['contas_excluidas_acumuladas']
df_merged['contas_ativas_comeco_mes'] = df_merged['contas_ativas_fim_mes'] + df_merged['contas_excluidas']
df_merged['churn'] = 100 * df_merged['contas_excluidas'] / df_merged['contas_ativas_comeco_mes'] 

In [39]:
df_merged.to_csv('./data/transformed/calculo_churn.csv', sep=',')

In [40]:
df_merged

,estado,ano_mes,contas_criadas,contas_excluidas,contas_criadas_acumuladas,contas_excluidas_acumuladas,contas_ativas_fim_mes,contas_ativas_comeco_mes,churn
1413,AC,201602,5.0,0.0,5.0,0.0,5.0,5.0,0.000000
972,AC,201603,5.0,0.0,10.0,0.0,10.0,10.0,0.000000
1022,AC,201604,8.0,0.0,18.0,0.0,18.0,18.0,0.000000
1414,AC,201605,4.0,0.0,22.0,0.0,22.0,22.0,0.000000
30,AC,201606,9.0,0.0,31.0,0.0,31.0,31.0,0.000000
...,...,...,...,...,...,...,...,...,...
1194,TO,202011,7.0,4.0,390.0,22.0,368.0,372.0,1.075269
1339,TO,202012,4.0,1.0,394.0,23.0,371.0,372.0,0.268817
1198,TO,202101,8.0,1.0,402.0,24.0,378.0,379.0,0.263852
1173,TO,202102,7.0,0.0,409.0,24.0,385.0,385.0,0.000000
